# Chapter 3 Lisp Overview

In [1]:
;; exercise 3.1
((lambda (x) 
  ((lambda (y) (+ y x)) 
   (* x x)))  ; y = x * x
 6) ; x=6

42

In [2]:
;; iterations - dolist
(defun length1 (list)
    (let ((len 0))
        ;; NOTE: the assignment element (_) is not used
        (dolist (_ list) (incf len)) 
        len))

(length1 '(1 2 3))

LENGTH1

3

In [3]:
;; iterations - mapc
(defun length2 (list)
    (let ((len 0))
        (mapc #'(lambda (_) (incf len)) list)
        len))

(length2 '(a b c d))

LENGTH2

4

SB-INT:SIMPLE-STYLE-WARNING: The variable _ is defined but never used.


In [4]:
;; iterations - do
(defun length3 (list)
    (do 
        ;; iteration forms
        ((len 0 (+ len 1))
         (l list (rest l)))

        ;; exit form
        ((null l) len)
        
        )) 

(length3 '(a b c d))

LENGTH3

4

In [5]:
;; iteratinos - loop
;; this is the most complex one

(defun length4 (list)
    (loop for element in list
        count t))

(length4 '(a b c d))

(defun length5 (list)
    (loop for element in list
        summing 1))

(length5 '(a b c d))

;; What this kind of expression is needed?
(defun length6 (list)
    (loop with len = 0
        until (null list)
        for element = (pop list)
        do (incf len)
        finally (return len)))

(length6 '(a b c d))

LENGTH4

4

LENGTH5

4

LENGTH6

4

In [6]:
(defun length7 (list)
    (count-if #'true list))
(defun true (x) t)

(length7 '(a b c d))

(defun length8 (list)
    (+ 1 (position-if #'true list :from-end t)))

(length8 '(a b c d))

LENGTH7

TRUE

4

LENGTH8

4

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::TRUE
SB-INT:SIMPLE-STYLE-WARNING: The variable X is defined but never used.


In [7]:
;; function accept keywords or derived version with predicate
;; (describe 'remove) 
(remove 1 '(1 2 3 2 1 0 -1))
(remove 1 '(1 2 3 2 1 0 -1) :key #'abs)
(remove 1 '(1 2 3 2 1 0 -1) :test #'<)
(remove 1 '(1 2 3 2 1 0 -1) :start 4)

(remove-if #'oddp '(1 2 3 2 1 0 -1))
(find-if #'oddp '(1 2 3 2 1 0 -1))

(2 3 2 0 -1)

(2 3 2 0)

(1 1 0 -1)

(1 2 3 2 0 -1)

(2 2 0)

1

In [8]:
;; loop as recursion
(defun length9 (list)
    (if (null list) 0 (+ 1 (length9 (rest list)))))

(length9 '(a b c d))

LENGTH9

4

In [9]:
;; tail recursion
(defun length10 (list)
    (length10-aux list 0))
(defun length10-aux (sublist len-so-far)
    (if (null sublist) len-so-far
        (length10-aux (rest sublist) (+ 1 len-so-far))))

(length10 '(a b c d))

LENGTH10

LENGTH10-AUX

4

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::LENGTH10-AUX


In [10]:
;; optional parameter
(defun length11 (list &optional (len-so-far 0))
    (if (null list) len-so-far (length11 (rest list) (+ 1 len-so-far))))

(length11 '(a b c d))

LENGTH11

4

In [11]:
;; defining local function
(defun length12 (list)
    (labels ((length13 (sublist len-so-far)
                (if (null sublist) len-so-far
                    (length13 (rest sublist) (+ 1 len-so-far)))))

        (length13 list 0)))

(length12 '(a b c d))

LENGTH12

4

In [12]:
; macro and back quotation
(defmacro while (test &rest body)
"Repeat body while test is true."
    `(loop (unless ,test (return nil))
    ,@body))

(macroexpand-1 '(while (< i 10) (setf i (+ i 1))))

WHILE

(LOOP (UNLESS (< I 10) (RETURN NIL))
      (SETF I (+ I 1)))

T

## Functions on list

In [13]:
;; exercise 3.3, print dotted notation of list
(defun print-dot (expr)
        (cond 
            ((null expr) (princ "nil"))
            ((consp expr) 
                (princ "(")
                (print-dot (first expr))
                (pr-rest (rest expr))
                (princ ")"))
            (t (princ expr))))

;; Think about two step recursion
(defun pr-rest (x)
    (princ " . ")
    (print-dot x))

(print-dot '(a b))
(print-dot '(defun print-dot (expr) (prog (princ (first expr)) (princ ".") (print-dot (rest expr)))))

PRINT-DOT

PR-REST

")"

")"

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::PR-REST
(A . (B . nil))(DEFUN . (PRINT-DOT . ((EXPR . nil) . ((PROG . ((PRINC . ((FIRST . (EXPR . nil)) . nil)) . ((PRINC . (. . nil)) . ((PRINT-DOT . ((REST . (EXPR . nil)) . nil)) . nil)))) . nil))))

In [14]:
;; exercise 3.4 similar to print, reuse the same print above
;; NOTE: how to reason on two step recursion?
(defun pr-rest (x)
    (cond ((null x))
          ((atom x) (princ " . ") (princ x))
          (t (princ " ") (print-dot (first x)) (pr-rest (rest x)))))

; TODO: detect the last list is cons or not
(print-dot (cons 'a (cons 1 nil)))
(print-dot (cons 'a (cons 1 2)))
(print-dot '(defun print-dot (expr) (prog (princ (first expr)) (princ ".") (print-dot (rest expr)))))
(print '(defun print-dot (expr) (prog (princ (first expr)) (princ ".") (print-dot (rest expr)))))

PR-REST

")"

")"

")"

(DEFUN PRINT-DOT (EXPR)
  (PROG (PRINC (FIRST EXPR)) (PRINC ".") (PRINT-DOT (REST EXPR))))

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PR-REST in DEFUN
(A 1)(A 1 . 2)(DEFUN PRINT-DOT (EXPR) (PROG (PRINC (FIRST EXPR)) (PRINC .) (PRINT-DOT (REST EXPR))))
(DEFUN PRINT-DOT (EXPR)
  (PROG (PRINC (FIRST EXPR)) (PRINC ".") (PRINT-DOT (REST EXPR)))) 

In [15]:
;; 3.5 equality
(defvar equality '(eq eql equal equalp))  ;  = tree-equal char-equal string-equal
(defvar x 42)
(defvar obj1 '(x 0 (x) "xy" "Xy" 0   0))
(defvar obj2 '(x 0 (x) "xy" "xY" 0.0 1))

(mapcar #'(lambda (eq_fn) 
    (format t "~%")
    (mapcar #'(lambda (x y) (format t "~a(~s, ~s) = ~b ~%" eq_fn x y (funcall eq_fn x y)))
        obj1 obj2)
) equality)

EQUALITY

X

OBJ1

OBJ2

((NIL NIL NIL NIL NIL NIL NIL) (NIL NIL NIL NIL NIL NIL NIL)
 (NIL NIL NIL NIL NIL NIL NIL) (NIL NIL NIL NIL NIL NIL NIL))


EQ(X, X) = T 
EQ(0, 0) = T 
EQ((X), (X)) = NIL 
EQ("xy", "xy") = NIL 
EQ("Xy", "xY") = NIL 
EQ(0, 0.0) = NIL 
EQ(0, 1) = NIL 

EQL(X, X) = T 
EQL(0, 0) = T 
EQL((X), (X)) = NIL 
EQL("xy", "xy") = NIL 
EQL("Xy", "xY") = NIL 
EQL(0, 0.0) = NIL 
EQL(0, 1) = NIL 

EQUAL(X, X) = T 
EQUAL(0, 0) = T 
EQUAL((X), (X)) = T 
EQUAL("xy", "xy") = T 
EQUAL("Xy", "xY") = NIL 
EQUAL(0, 0.0) = NIL 
EQUAL(0, 1) = NIL 

EQUALP(X, X) = T 
EQUALP(0, 0) = T 
EQUALP((X), (X)) = T 
EQUALP("xy", "xy") = T 
EQUALP("Xy", "xY") = T 
EQUALP(0, 0.0) = T 
EQUALP(0, 1) = NIL 


In [16]:
;; 3.6 tables
;; 3.6.1 associate list, use pair to form linked list, ((k_1 . v_1) (k_2 . v_2) ... (k_n . v_n))
;; O(n)
(defvar state-table
    '((AL . Alabama) (AK . Alaska) (AZ . Arizona) (AR . Arkansas)))
(assoc 'AL state-table)
(rassoc 'Alaska state-table)

STATE-TABLE

(AL . ALABAMA)

(AK . ALASKA)

In [17]:
;; 3.6.2 hash table
(defvar table (make-hash-table))
(setf (gethash 'AL table) 'Alabama)
(setf (gethash 'AK table) 'Alaska)
(setf (gethash 'AZ table) 'Arizona)
(setf (gethash 'AR table) 'Arkansas)

(gethash 'AR table)

;; remhash remove a key
(remhash 'AR table)
(gethash 'AR table)

;; clrhash removes all pairs
(clrhash table)
(gethash 'AZ table)

TABLE

ALABAMA

ALASKA

ARIZONA

ARKANSAS

...

NIL

NIL

#<HASH-TABLE :TEST EQL :COUNT 0 {10036019B3}>

NIL

NIL

In [18]:
;; 3.6.3 property list: (k_1  v_1 k_2 v_2 ... k_n v_n)
;; every symbol already has plist
(setf (get 'AL 'state) 'Alabama)
(setf (get 'AK 'state) 'Alaska)
(setf (get 'AZ 'state) 'Arizona)
(setf (get 'AR 'state) 'Arkansas)

(get 'AR 'state)
(symbol-plist 'AR)

ALABAMA

ALASKA

ARIZONA

ARKANSAS

ARKANSAS

(STATE ARKANSAS)

In [19]:
;; 3.7 trees
(defvar tree '((ab)((c))(de)))
;; deep copy and tree equality
(tree-equal tree (copy-tree tree))

;; subst one 
(subst 'new 'old '(old (very old)))
;; subst using a list of pairs
(sublis '((old . new) (very . greatly))
    '(old (very old)))

TREE

T

(NEW (VERY NEW))

(NEW (GREATLY NEW))

In [20]:
;; 3.10 destructive function
;; exercise 3.5 
;; TODO: guesser program

In [21]:

;; 3.14 antibugging tool
(defun sqr (x)
    (assert (numberp x) (x))
    (* x x))

(defun eat-porridge (bear)
    (assert (< too-cold (temperature (bear-porridge bear)) too-hot)
            (bear (bear-porridge bear))
            "~a's porridge is not just right: ~a"
            bear (hotness (bear-porridge bear)))
    (eat (bear-porridge bear)))

SQR

EAT-PORRIDGE

SB-INT:SIMPLE-STYLE-WARNING: undefined function: (COMMON-LISP:SETF COMMON-LISP-USER::BEAR-PORRIDGE)
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::BEAR-PORRIDGE
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::EAT
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::HOTNESS
SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::TEMPERATURE
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TOO-COLD
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::TOO-HOT


In [22]:
;; Exercise 3.6 
(setf a 'global-a)
(defvar *b* 'global-b)
(defun fn () *b*)

(let ((a 'local-a)
      (*b* 'local-b))
      (list a *b* (fn) (symbol-value 'a) (symbol-value '*b*)))
;   my answer:      'local-a, 'local-b 'global-b, 'global-a, 'global-b
;   correct answer: 'local-a, 'local-b 'local-b,  'global-a, 'local-b
;
;   Why?
;   *b* is a declared to be dynamic scoped variable, whose value depends on the context
;   (fn) => 'local-b
;   (symbol-value '*b') => 'local-b

GLOBAL-A

*B*

FN

(LOCAL-A LOCAL-B LOCAL-B GLOBAL-A LOCAL-B)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A


In [23]:
; multiple values bind
(round 5.1)
(defun show-both (x)
    (multiple-value-bind (int rem)
        (round x)
        (format t "~f = ~d + ~f ~%" x int rem)))

(show-both 5.1)

(values 1 2 3) ;; compose multiple values returns

5

0.099999905

SHOW-BOTH

NIL

1

2

3

5.1 = 5 + 0.099999905 


In [24]:
;; parameter 
(defun problem (x op y)
    "Ask a single math problem then check the answer"
    (format t "~&How much is ~d ~a ~d? ~%" x op y)
    (let ((answer (read))
            (correct-answer (funcall op x y)))
            (if (= answer correct-answer)
                    (format t "Correct! The answer is ~f." correct-answer)
                    (format t "Sorry the answer is not correct, it should be ~f." correct-answer))))

(defun make-quiz (op range n)
    "Ask a user a series of math problem."
    (dotimes (i n)
        (problem (random range) op (random range))))

; optional position dependent parameters
(defun make-quiz (&optional (op '+) (range 100) (n 10))
    "Ask a user a series of math problem."
    (dotimes (i n)
        (problem (random range) op (random range))))

; optional position independent parameters
(defun make-quiz (&key (op '+) (range 100) (n 10))
    "Ask a user a series of math problem."
    (dotimes (i n)
        (problem (random range) op (random range))))

PROBLEM

MAKE-QUIZ

MAKE-QUIZ

MAKE-QUIZ

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-QUIZ in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::MAKE-QUIZ in DEFUN


In [25]:
;; using keyword with built-in functions
(setf a '(1 2 3 4 -5 6.0))
(find 5 a :key #'abs)
(find 6 a :test #'equalp)
(find 4 a :test #'<)

(1 2 3 4 -5 6.0)

-5

6.0

6.0

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A


In [26]:
;; define new function to find all
(defvar nums '(1 -1 2 3 4 -5 6.0))

;; assign the function body to the symbol function slot
(setf (symbol-function 'find-all-if) #'remove-if-not)
(find-all-if (lambda (x) (< x 4)) nums :key #'abs)

(defun complement1 (fn)
    ;; TODO: whether to use #' or just lambda?
    (lambda (&rest args) (not (apply fn args))))

(apply (complement1 #'equalp) '(1 2))

(defun find-all (item sequence &rest keyword-args 
                 &key (test #'eql) test-not &allow-other-keys)
    (if test-not
        (apply #'remove item sequence :test-not (complement1 test-not) keyword-args)
        (apply #'remove item sequence :test (complement1 test) keyword-args)))

(find-all 4 nums :test #'<= :key #'abs)

NUMS

#<FUNCTION REMOVE-IF-NOT>

(1 -1 2 3)

COMPLEMENT1

T

FIND-ALL

(4 -5 6.0)

In [27]:
;; exercise 3.7
;; The keyword is evaluated using AND logic, so the leftmost value is selected
;; Reasons:
;; - first to evaluation
;; - efficient to override

;; exercise 3.8
;; Remove :test and :test-not entry in `keyword-args` before passing to remove
(defun find-all (item sequence &rest keyword-args 
                 &key (test #'eql) test-not &allow-other-keys)

    (let* ((rem-keyword-args (remove-pair :test keyword-args))
           (rem-keyword-args (remove-pair :test-not rem-keyword-args)))
        (if test-not
                (apply #'remove item sequence :test-not (complement1 test-not) rem-keyword-args)
                (apply #'remove item sequence :test (complement1 test) rem-keyword-args))))

(defun remove-pair (key sequence)
    "Remove a pair with key val in list (key1, val1, key2, val2, ... )"
    (cond ((null sequence) nil)
          ((eq key (first sequence)) (remove-pair key (rest (rest sequence))))
          (t (cons (first sequence) (cons (second sequence) (remove-pair key (rest (rest sequence))))))))

(remove-pair :test '(:test 'a :test 'b 2 'c))

(setf nums '(1 -1 2 3 4 1.0 1))
(find-all 1 nums :test #'eq :key #'abs)

FIND-ALL

REMOVE-PAIR

(2 'C)

(1 -1 2 3 4 1.0 1)

(1 -1 1)

SB-INT:SIMPLE-STYLE-WARNING: undefined function: COMMON-LISP-USER::REMOVE-PAIR
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::FIND-ALL in DEFUN


In [57]:
;; exercise 3.9, length using reduce
(defun length15 (list)
    (reduce (lambda (x _) (+ 1 x)) list :initial-value 0))

(length15 a)

LENGTH15

6

SB-INT:SIMPLE-STYLE-WARNING: The variable _ is defined but never used.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::LENGTH15 in DEFUN
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::A


In [34]:
;; exercise 3.10
; (describe 'lcm)
(lcm 20 25 30 100)

; (load "draw-cons-tree.lisp")
; (use-package :draw-cons-tree)

;; destructive concatenation, reverse and concate in place
; (describe 'nreconc)
;                  -- foo
;  --res           |       -- bar
;  |               |       | 
; [o|o]---[o|o]---[o|o]---[o|o]---[o|o]---[o|/]
;  |       |       |       |       |       |      
;  C       B       A       E       F       G 

(let* ((foo '(a b c))
       (bar '(e f g))
       (res (nreconc foo bar))) 
    (princ res)
    (princ foo)
    (princ bar))

300

(E F G)

SB-INT:CONSTANT-MODIFIED: Destructive function NRECONC called on constant data: (A
                                                                                                 B
                                                                                                 C)
See also:
  The ANSI Standard, Special Operator QUOTE
  The ANSI Standard, Section 3.7.1
(C B A E F G)(A E F G)(E F G)

In [37]:
;; exercise 3.11
(setq alist '((a . 1) (b . 2) (c . 3)))
(acons 'd 4 alist)

((A . 1) (B . 2) (C . 3))

((D . 4) (A . 1) (B . 2) (C . 3))

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ALIST
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::ALIST


In [73]:
;; exercise 3.12 sentence print
(defun print-sentence (word-list)
    (format t "~a" (string-capitalize (first word-list)))
    (mapcar (lambda (word) (format t " ~a" (string-downcase word))) (rest word-list))
    (princ "."))

(print-sentence '(hello world))

(format t "~%")
(format t "~@(~{~a~^ ~}.~)" '(this is a test))

PRINT-SENTENCE

"."

NIL

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::PRINT-SENTENCE in DEFUN
Hello world.
This is a test.